In [1]:
import models as m
import os
print(os.getcwd())
maindir=os.getcwd() 

/home/jonah/CLionProjects/oxdna/oxDNA/ANM_EXAMPLES


# Welcome! In this tutorial we will make a hybrid DNA cage ligated to KDPG Aldolase

## We will use the KDPG files generated in the KDPG Example Notebook as the protein

## Also we have a pre-relaxed DNA cage structure we provide

## Here is our pre-relaxed DNA cage

![dna cage](./examples/imgs/dcage.png)


## And our KDPG Aldolase
    
    In pink are the Cysteine residues used to ligate the DNA to the protein

![kdpg ligate](./examples/imgs/kdpgligate.png)

## NOTE: Mutual traps between the ends of each arm were added during the relaxtion to get the cage into approximately the right shape


# Combining the Structures

    !!!! MUST BE DONE IN THE FOLLOWING ORDER TO GET CORRECT SIMULATION FILES !!!!
    
    Drag and drop 'dnacage' .top and .dat files into oxView
    Drag and drop 'kdpg' .top and .dat files into oxView
    
    We have:
![cage assembly](./examples/imgs/cageassemble.png)


## Now we select the protein system in oxView, translate and rotate it to have the modified residues as close to the DNA arms as we can get (doesn't have to be perfect)

SELECTED:

![cage mov](./examples/imgs/cageselect.png)


ROTATED and TRANSLATED:

![rot and tran](./examples/imgs/cageplaced.png)

## Drop the kdpg.par file in and download the simulation files

## Reopen the simulation files in a fresh instance of oxView

### To finish setting up the linker between the DNA and protein we sequentially select each modified protein residue then the DNA base it is linked to


![cage trap](./examples/imgs/cagetrap.png)


### Simply Click 'Selected Base Trap File' to download the mutual trap file

    This is our mutual trap file
    
![trap file](./examples/imgs/trapfile.png)


### We now edit the r0 (equilbrium distance) and stiff (force constant) parameters to match fully atomistic simulations of our linker (LC-SPDP)

    (We also take into account length of the amine modification on the DNA
    and the cysteine side length)

    Edited Trap file
    
![etrap file](./examples/imgs/etrapfile.png)


## Our System is ready to be relaxed

### Using the Provided Input Files we will relax the structure using Monte Carlo Methods
    
    Monte Carlo is used to prevent explosive forces from the added mutual traps. Once the traps are relaxed
    we can use MD methods to quickly relax the rest of the system

    We will inspect the system visually to check if system is relaxed according to the following criteria
    1. DNA arms should be very close to their respective modified protein residues
    2. DNA arms should have fully formed Duplexes (or close to it)

In [8]:
# Monte Carlo Sim

wdir = os.getcwd()

os.chdir('./examples/Cage/')

!/home/jonah/CLionProjects/oxdna/oxDNA/build/bin/oxDNA ./input_relaxMC

os.chdir(wdir) # If this isn't called use os.chdir(maindir) to return to your original folder

           0  -0.575795   0.000  0.000  0.000  
        1000  -0.471079   0.404  0.719  0.000  
        2000  -0.475207   0.405  0.721  0.000  
        3000  -0.474535   0.405  0.721  0.000  
        4000  -0.472602   0.405  0.721  0.000  
        5000  -0.467709   0.406  0.721  0.000  
        6000  -0.462515   0.406  0.721  0.000  
        7000  -0.469383   0.406  0.722  0.000  
        8000  -0.473366   0.406  0.722  0.000  
        9000  -0.481467   0.406  0.722  0.000  
       10000  -0.475703   0.406  0.722  0.000  


## It hasn't been long but let's take a look!

![mc relax](./examples/imgs/cagemc.png)

## Good Enough for an MD simulation! Let's use our CUDADNANM Interaction to Relax the structure

In [9]:
# CUDA MD Relax

wdir = os.getcwd()

os.chdir('./examples/Cage/') # Directory containing Topology, Parameter, Dat, force Files

!/home/jonah/CLionProjects/oxdna/oxDNA/build/bin/oxDNA ./input_cuda

os.chdir(wdir) # If this isn't called use os.chdir(maindir) to return to your original folder

           0         0.0000  -0.484209   0.210801  -0.273408 
       10000        20.0000  -0.466222   0.203609  -0.262613 
       20000        40.0000  -0.460448   0.204456  -0.255991 
       30000        60.0000  -0.464303   0.217320  -0.246983 
       40000        80.0000  -0.457388   0.207596  -0.249792 
       50000       100.0000  -0.452555   0.200559  -0.251997 
       60000       120.0000  -0.456037   0.200784  -0.255253 
       70000       140.0000  -0.457993   0.200202  -0.257791 
       80000       160.0000  -0.461624   0.207330  -0.254294 
       90000       180.0000  -0.464548   0.213289  -0.251260 
      100000       200.0000  -0.465425   0.206583  -0.258842 


## Looks Good! Let's try a couple of analyses from our script package!

    Make sure you have downloaded https://github.com/sulcgroup/oxdna_analysis_tools
    
    To Use the tools activate any (Python 3) env of your choice:
        
        If a script is missing any dependencies, install the missing packages
        
## We will try 2 basic Analyses on the Trajectory we just generated 

    1. Compute Mean Structure and Means Squared Deviations
    2. Means Squared Deviations from the previously computed mean structure
    
# 1. Compute Mean Structure

In [12]:
mydir = '/home/jonah/oxdna_analysis_tools/' # Replace with your oxdna_analysis_tools directory

import subprocess as sp 
import os

wdir = os.getcwd()
os.chdir(wdir+'/examples/Cage/')

# Call to generate Mean Structures
sp.check_call(mydir+'compute_mean.py -f json trajectory.dat hcage.top'
              + ' -o cagemean.json', shell=True)

sp.check_call(mydir+'compute_mean.py -f oxdna trajectory.dat hcage.top'
              + ' -o cagemean.dat', shell=True)

os.chdir(wdir) # If this isn't called use os.chdir(maindir) to return to your original folder

## We have generated two mean structures, a json format for calculating deviations and an oxdna format for visualization

    Dragging and dropping 'hcage.top' and 'cagemean.dat' into oxView we see the mean structure:
    
![cage mean](./examples/imgs/cagemean.png)

    Don't close this Tab! We will need this structure again shortly!

### It's pretty wonky because of the very short simulation time, for better sampling we would need to run many more steps. A good measure of steps for proper sampling is whether a duplicate simulation returns the same mean energy and std. deviation.

  
# 2. Compute Deviations

    Now we will use the 'mean.json' file to calculate the mean squared deviation of each particle from it's 
    mean position


In [4]:
mydir = '/home/jonah/oxdna_analysis_tools/' # Replace with your oxdna_analysis_tools directory

import subprocess as sp 
import os

wdir = os.getcwd()
os.chdir(wdir+'/examples/Cage/')

# Call to generate Deviations
sp.check_call(mydir+'compute_deviations.py cagemean.json trajectory.dat hcage.top'
              + ' -o cagedevs.json', shell=True)

os.chdir(wdir) # If this isn't called use os.chdir(maindir) to return to your original folder

## Now we can drap and drop 'cagedevs.json' onto our structure to view the deviations, This is what we see:

![cage devs](./examples/imgs/cagedevs.png)

### Check out the rest of our script to take a look at what you can do! The supporting libraries are well documented for your use as well!